In [1]:
import os
os.environ

environ{'LC_PAPER': 'nb_NO.UTF-8',
        'PROJ_LIB': '/home/vagrant/Miniconda3-4.5.1-Linux-x86_64/envs/py3openwind/share/proj',
        '_CONDA_SET_PROJ_LIB': '/home/vagrant/anaconda/envs/py3openwind/share/proj',
        'LC_ADDRESS': 'nb_NO.UTF-8',
        'XDG_SESSION_ID': '3',
        'LC_MONETARY': 'nb_NO.UTF-8',
        'SHELL': '/bin/bash',
        'TERM': 'xterm-color',
        'SSH_CLIENT': '10.0.2.2 37308 22',
        'CONDA_SHLVL': '1',
        'CONDA_PROMPT_MODIFIER': '(py3openwind) ',
        'LC_NUMERIC': 'nb_NO.UTF-8',
        'SSH_TTY': '/dev/pts/0',
        'LC_ALL': 'en_US.UTF-8',
        'PYTHONBREAKPOINT': 'ipdb.set_trace',
        'USER': 'vagrant',
        'LC_TELEPHONE': 'nb_NO.UTF-8',
        'LS_COLORS': 'rs=0:di=01;34:ln=01;36:mh=00:pi=40;33:so=01;35:do=01;35:bd=40;33;01:cd=40;33;01:or=40;31;01:mi=00:su=37;41:sg=30;43:ca=30;41:tw=30;42:ow=34;42:st=37;44:ex=01;32:*.tar=01;31:*.tgz=01;31:*.arc=01;31:*.arj=01;31:*.taz=01;31:*.lha=01;31:*.lz4=01;31:*.lzh=01;31:*.

In [3]:
import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'project.settings'

import django
django.setup()

from django.conf import settings

RuntimeError: populate() isn't reentrant

In [ ]:
from openwind import sar_wind

In [4]:
from geo_spaas_wind.utils import wind_from_sar_and_arome_forecast

In [5]:
fn = 'http://nbstds.met.no/thredds/dodsC/NBS/S1B/2019/06/30/IW/S1B_IW_GRDM_1SDV_20190630T051108_20190630T051141_016921_01FD7F_21CF.nc'

In [6]:
from nansat.nansat import Nansat

In [7]:
fn

'http://nbstds.met.no/thredds/dodsC/NBS/S1B/2019/06/30/IW/S1B_IW_GRDM_1SDV_20190630T051108_20190630T051141_016921_01FD7F_21CF.nc'